In [105]:
#import midi_functions
import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

#import pydot as pyd
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [69]:
song = './Bach-1.mid'
features = midi_functions.get_features(path=song)

input_length = 10 #How many past notes we want to give the LSTM to predict the next one
X, Y = midi_functions.get_XY(features,n_notes=input_length)
X = X[:,np.arange(0,4 * input_length,4)]#gets only the notes and discards velocity/time info
Y = Y[:,0]

#How many notes are present?
sorted_notes = np.copy(Y)
sorted_notes.sort()
sorted_notes = np.unique(sorted_notes)

number_of_different_notes = len(sorted_notes)
print('Number of different notes: {}'.format(number_of_different_notes))

#Convert Y into the one-hot-representation because I think that is the only thing that works with softmax representation
for i,note in enumerate(Y): #make all the notes a number between 0 and number_of_different_notes
    Y[i] = np.where(sorted_notes == note)[0]
Y = np_utils.to_categorical(Y)
print(Y.shape)


Number of different notes: 38
(699, 38)


In [37]:
#randomize the two arrays and split into training and validation
n = len(X)
permutation = np.random.permutation(n)
X = X[permutation]
Y = Y[permutation]

X_val = X[:200]
Y_val = Y[:200]
X_train = X[200:]
Y_train = Y[200:]

In [120]:
X = np.loadtxt('X.txt')
Y = np.loadtxt('Y.txt')

n = len(X)
permutation = np.random.permutation(n)
X = X[permutation]
Y = Y[permutation]

sorted_notes = np.copy(Y)
sorted_notes.sort()
sorted_notes = np.unique(sorted_notes)

number_of_different_notes = len(sorted_notes)
print('Number of different notes: {}'.format(number_of_different_notes))

for i,note in enumerate(Y): #make all the notes a number between 0 and number_of_different_notes
    Y[i] = np.where(sorted_notes == note)[0]
Y = np_utils.to_categorical(Y)
print(X.shape)

#X_val = X[:1000]
#Y_val = Y[:1000]
#X_train = X[1000:]
#Y_train = Y[1000:]
#print(X_train.shape)

Number of different notes: 67
(9976, 10)


In [122]:
input_length = 10
#input_dimensions = (input_length,1)

n_hidden_units = number_of_different_notes #this should probably be the number of different notes since it is also the output of softmax

model = Sequential()
model.add(LSTM(
    n_hidden_units,
    input_shape=((input_length,1)),
    return_sequences=True
))
model.add(LSTM(n_hidden_units, return_sequences=False))
#model.add(Dropout(0.3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
# Load the weights to each node
#model.load_weights('weights.hdf5')


#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
print(model.summary())
#print(model.get_weights())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_27 (LSTM)               (None, 10, 67)            18492     
_________________________________________________________________
lstm_28 (LSTM)               (None, 67)                36180     
_________________________________________________________________
activation_15 (Activation)   (None, 67)                0         
Total params: 54,672
Trainable params: 54,672
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
filepath = "./weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(X.reshape(9976, 10,1), Y, validation_split=0.33,epochs=200, batch_size=64, callbacks=callbacks_list)

Train on 6683 samples, validate on 3293 samples
Epoch 1/200
6683/6683 [==============================] - 2s 267us/step - loss: 2.9837 - acc: 0.2060 - val_loss: 3.2074 - val_acc: 0.1892
Epoch 2/200
6683/6683 [==============================] - 2s 268us/step - loss: 2.9826 - acc: 0.2048 - val_loss: 3.2767 - val_acc: 0.1661
Epoch 3/200
6683/6683 [==============================] - 2s 272us/step - loss: 2.9815 - acc: 0.2066 - val_loss: 3.2270 - val_acc: 0.1746
Epoch 4/200
6683/6683 [==============================] - 2s 273us/step - loss: 2.9849 - acc: 0.2105 - val_loss: 3.2072 - val_acc: 0.1761
Epoch 5/200
6683/6683 [==============================] - 2s 271us/step - loss: 2.9779 - acc: 0.2098 - val_loss: 3.1973 - val_acc: 0.1698
Epoch 6/200
6683/6683 [==============================] - 2s 274us/step - loss: 2.9817 - acc: 0.2069 - val_loss: 3.2153 - val_acc: 0.1613
Epoch 7/200
6683/6683 [==============================] - 2s 269us/step - loss: 2.9763 - acc: 0.2065 - val_loss: 3.2175 - val_acc: 

6683/6683 [==============================] - 2s 299us/step - loss: 2.9291 - acc: 0.2352 - val_loss: 3.1900 - val_acc: 0.1947
Epoch 61/200
6683/6683 [==============================] - 2s 303us/step - loss: 2.9290 - acc: 0.2333 - val_loss: 3.1929 - val_acc: 0.2026
Epoch 62/200
6683/6683 [==============================] - 2s 303us/step - loss: 2.9282 - acc: 0.2336 - val_loss: 3.1923 - val_acc: 0.1959
Epoch 63/200
6683/6683 [==============================] - 2s 304us/step - loss: 2.9281 - acc: 0.2390 - val_loss: 3.1943 - val_acc: 0.1928
Epoch 64/200
6683/6683 [==============================] - 2s 301us/step - loss: 2.9239 - acc: 0.2361 - val_loss: 3.1887 - val_acc: 0.1995
Epoch 65/200
6683/6683 [==============================] - 2s 301us/step - loss: 2.9238 - acc: 0.2441 - val_loss: 3.2087 - val_acc: 0.1922
Epoch 66/200
6683/6683 [==============================] - 2s 301us/step - loss: 2.9222 - acc: 0.2373 - val_loss: 3.2099 - val_acc: 0.1752
Epoch 67/200
6683/6683 [=======================

6683/6683 [==============================] - 2s 289us/step - loss: 2.8885 - acc: 0.2719 - val_loss: 3.1930 - val_acc: 0.2098
Epoch 120/200
6683/6683 [==============================] - 2s 286us/step - loss: 2.8885 - acc: 0.2744 - val_loss: 3.2064 - val_acc: 0.1968
Epoch 121/200
6683/6683 [==============================] - 2s 289us/step - loss: 2.8912 - acc: 0.2728 - val_loss: 3.1992 - val_acc: 0.2004
Epoch 122/200
6683/6683 [==============================] - 2s 295us/step - loss: 2.8855 - acc: 0.2735 - val_loss: 3.2270 - val_acc: 0.1986
Epoch 123/200
6683/6683 [==============================] - 2s 297us/step - loss: 2.8887 - acc: 0.2722 - val_loss: 3.1867 - val_acc: 0.2095
Epoch 124/200
6683/6683 [==============================] - 2s 295us/step - loss: 2.8872 - acc: 0.2746 - val_loss: 3.2139 - val_acc: 0.2092
Epoch 125/200
6683/6683 [==============================] - 2s 283us/step - loss: 2.8853 - acc: 0.2734 - val_loss: 3.1945 - val_acc: 0.2159
Epoch 126/200
6683/6683 [================

6683/6683 [==============================] - 2s 282us/step - loss: 2.8623 - acc: 0.2949 - val_loss: 3.1903 - val_acc: 0.2202
Epoch 179/200
6683/6683 [==============================] - 2s 282us/step - loss: 2.8630 - acc: 0.2933 - val_loss: 3.2061 - val_acc: 0.2038
Epoch 180/200
6683/6683 [==============================] - 2s 291us/step - loss: 2.8569 - acc: 0.2999 - val_loss: 3.2125 - val_acc: 0.2086
Epoch 181/200
6683/6683 [==============================] - 2s 296us/step - loss: 2.8608 - acc: 0.2984 - val_loss: 3.1844 - val_acc: 0.2211
Epoch 182/200
6683/6683 [==============================] - 2s 294us/step - loss: 2.8574 - acc: 0.2952 - val_loss: 3.1862 - val_acc: 0.2268
Epoch 183/200
2368/6683 [=========>....................] - ETA: 1s - loss: 2.8542 - acc: 0.3057

In [128]:
#Generate 200 notes

#random sequence to initialize predictions
init = X[np.random.randint(0,len(X))]

predicted_notes = np.zeros(200 + input_length)
predicted_notes[0:input_length] = init

for note_index in range(200):
    x = predicted_notes[note_index:note_index + input_length]
    print('Input: {}'.format(x))
    prediction = model.predict(x.reshape(1,10,1))
    print('Prediction: {}'.format(np.argmax(prediction)))
    predicted_notes[note_index + input_length] = sorted_notes[np.argmax(prediction)]

Input: [52. 57. 60. 64. 69. 71. 40. 52. 56. 64.]
Prediction: 39
Input: [57. 60. 64. 69. 71. 40. 52. 56. 64. 71.]
Prediction: 39
Input: [60. 64. 69. 71. 40. 52. 56. 64. 71. 71.]
Prediction: 39
Input: [64. 69. 71. 40. 52. 56. 64. 71. 71. 71.]
Prediction: 37
Input: [69. 71. 40. 52. 56. 64. 71. 71. 71. 69.]
Prediction: 37
Input: [71. 40. 52. 56. 64. 71. 71. 71. 69. 69.]
Prediction: 37
Input: [40. 52. 56. 64. 71. 71. 71. 69. 69. 69.]
Prediction: 40
Input: [52. 56. 64. 71. 71. 71. 69. 69. 69. 72.]
Prediction: 40
Input: [56. 64. 71. 71. 71. 69. 69. 69. 72. 72.]
Prediction: 35
Input: [64. 71. 71. 71. 69. 69. 69. 72. 72. 67.]
Prediction: 35
Input: [71. 71. 71. 69. 69. 69. 72. 72. 67. 67.]
Prediction: 35
Input: [71. 71. 69. 69. 69. 72. 72. 67. 67. 67.]
Prediction: 35
Input: [71. 69. 69. 69. 72. 72. 67. 67. 67. 67.]
Prediction: 31
Input: [69. 69. 69. 72. 72. 67. 67. 67. 67. 63.]
Prediction: 32
Input: [69. 69. 72. 72. 67. 67. 67. 67. 63. 64.]
Prediction: 32
Input: [69. 72. 72. 67. 67. 67. 67. 63. 

Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 65. 65. 65. 65. 65. 65.]
Prediction: 33
Input: [65. 65. 65. 65. 6

In [129]:
predicted_notes

array([52., 57., 60., 64., 69., 71., 40., 52., 56., 64., 71., 71., 71.,
       69., 69., 69., 72., 72., 67., 67., 67., 67., 63., 64., 64., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65.,
       65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65., 65